Importaciones al programa

In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Verificar que Tensorflow tenga cargada una GPU

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


Prepocesamiento de los datos 1

Variables iniciales

In [3]:
depth=3
width=256
height=256
train_path = '../data/dataset-2/train'
valid_path = '../data/dataset-2/valid'
default_image_size = tuple((width, height))

Funcion para leer y cargar cada imagen

In [4]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

Solo las carpetas 20 a 23 para probar con POTATO
Se asegura de agregar solamente imagenes terminadas en .jpg

In [5]:
image_list, label_list = [], []
try:
    plant_disease_folder_list = listdir(train_path)
    for plant_disease_folder in plant_disease_folder_list[20:23]:
        print(f"Processing {plant_disease_folder} ...")
        plant_disease_image_list = listdir(f"{train_path}/{plant_disease_folder}/")
        for image in plant_disease_image_list[:200]:
            image_directory = f"{train_path}/{plant_disease_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_disease_folder)
except Exception as e:
    print(f"Error : {e}")

Processing Potato___Early_blight ...
Processing Potato___healthy ...
Processing Potato___Late_blight ...


In [6]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

Codificacion de los Labels

In [7]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
print(f"Clases a analizar: {label_binarizer.classes_}")
print(f"Clases binarizadas: {label_binarizer}")

Clases a analizar: ['Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy']
Clases binarizadas: LabelBinarizer()


In [8]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)
print(x_train)

[[[[0.573  0.4978 0.5557]
   [0.5645 0.4888 0.547 ]
   [0.569  0.4934 0.5513]
   ...
   [0.742  0.676  0.6934]
   [0.76   0.6934 0.711 ]
   [0.8    0.7334 0.751 ]]

  [[0.6045 0.529  0.5864]
   [0.5913 0.5156 0.573 ]
   [0.5864 0.511  0.569 ]
   ...
   [0.8    0.7334 0.751 ]
   [0.782  0.7153 0.7334]
   [0.778  0.711  0.729 ]]

  [[0.5957 0.52   0.5776]
   [0.582  0.507  0.5645]
   [0.5776 0.5024 0.56  ]
   ...
   [0.7866 0.72   0.738 ]
   [0.76   0.6934 0.711 ]
   [0.742  0.676  0.6934]]

  ...

  [[0.4712 0.4268 0.4978]
   [0.4934 0.449  0.52  ]
   [0.511  0.4666 0.5376]
   ...
   [0.76   0.7246 0.7734]
   [0.76   0.7246 0.7734]
   [0.76   0.7246 0.7734]]

  [[0.5645 0.52   0.5913]
   [0.5776 0.533  0.6045]
   [0.5376 0.4934 0.5645]
   ...
   [0.7466 0.711  0.76  ]
   [0.76   0.7246 0.7734]
   [0.769  0.7334 0.782 ]]

  [[0.507  0.4622 0.533 ]
   [0.5156 0.4712 0.542 ]
   [0.4712 0.4268 0.4978]
   ...
   [0.7246 0.689  0.738 ]
   [0.742  0.7065 0.7554]
   [0.76   0.7246 0.7734]]]


 

Matriz aumentada

In [9]:
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest")

Definicion del modelo y configuracion de las layers

In [10]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights= 'imagenet', include_top=False, input_shape= (256,256,3))

out = model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=model.input, outputs=predictions)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

Definicion del modelo de optimizacion de Adam

In [12]:
INT=1e-4
EPOCHS= 10
op = Adam(learning_rate=INT, decay=0.5)
model.compile(loss="binary_crossentropy",optimizer=op,metrics=["accuracy"])

Entrenamiento del modelo

In [13]:
BS=22
history=model.fit(
    aug.flow(x_train,y_train,batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train),
    epochs=EPOCHS,verbose=1)

Epoch 1/10


Calculo de precision del digo

In [ ]:
print("Model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")